# TSA Chapter 4: Case Study - SARIMA Diagnostics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch4/TSA_ch4_case_diagnostics/TSA_ch4_case_diagnostics.ipynb)

Residual diagnostics for SARIMA model on airline data.

In [ ]:
!pip install numpy pandas matplotlib statsmodels -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Load airline passengers
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv',
                    parse_dates=['Month'], index_col='Month')
passengers = data['Passengers']
passengers.index.freq = 'MS'
log_passengers = np.log(passengers)

In [ ]:
model = SARIMAX(log_passengers, order=(0,1,1), seasonal_order=(0,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
fit = model.fit(disp=False); resid = fit.resid[13:]
fig, axes = plt.subplots(2, 2, figsize=(12, 7))
axes[0,0].plot(resid.index, resid.values, color=BLUE, lw=0.8); axes[0,0].axhline(0, color=GRAY, ls='--'); axes[0,0].set_title('Residuals', fontweight='bold')
axes[0,1].hist(resid.dropna(), bins=25, density=True, color=BLUE, alpha=0.7, edgecolor='white')
x = np.linspace(resid.min(), resid.max(), 100); axes[0,1].plot(x, stats.norm.pdf(x, resid.mean(), resid.std()), color=RED, lw=2)
axes[0,1].set_title('Histogram + Normal', fontweight='bold')
acf_r = acf(resid.dropna(), nlags=36); axes[1,0].bar(range(len(acf_r)), acf_r, color=BLUE, width=0.35)
ci = 1.96/np.sqrt(len(resid.dropna())); axes[1,0].axhline(ci, color=RED, ls='--'); axes[1,0].axhline(-ci, color=RED, ls='--')
axes[1,0].set_title('ACF Residuals', fontweight='bold')
stats.probplot(resid.dropna(), plot=axes[1,1]); axes[1,1].set_title('Q-Q Plot', fontweight='bold')
plt.tight_layout(); save_chart(fig, 'ch4_case_diagnostics'); plt.show()